In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.autograd import Variable
import torch.nn.functional as F

from torchvision import datasets, transforms
from utils import build_dataset, flops_to_string, params_to_string, accuracy
from tqdm import tqdm_notebook

import time
from tensorboardX import SummaryWriter
writer = SummaryWriter(f'runs/MobileNet_{int(time.time())}')

from nets.MobileNet_CIFAR import MobileNet_CIFAR

In [ ]:
class Solver(object):
    def __init__(self, model, dataset='MNIST', n_epochs=100, lr=0.001):
        self.n_epochs = n_epochs
        
        self.train_loader, self.test_loader = build_dataset(dataset, './data', batch_size = 128)
        
        self.image_dim = {'MNIST': 28*28, 'CIFAR10': 3*32*32}[dataset]
                
        self.net = model().cuda()
        
        self.loss_fn = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.net.parameters(), lr=lr)
                
    def train(self):
        self.net.train()
        self.best_loss = 999
        for epoch_i in tqdm_notebook(range(self.n_epochs)):
            epoch_loss = 0
            for batch_idx, (images, labels) in enumerate(self.train_loader):
                images = Variable(images).cuda()
                labels = Variable(labels).cuda()
                
                logits = self.net(images)
                
                loss = self.loss_fn(logits, labels)
                
                total_loss = loss

                self.optimizer.zero_grad()
                total_loss.backward()

                self.optimizer.step()
                
                epoch_loss += float(loss.data)    
            epoch_loss /= len(self.train_loader.dataset)
            # save best model
            if best_loss > epoch_loss:
                best_loss = epoch_loss
                torch.save(self.net.state_dict(), 'models/MobileNet_CIFAR_best_loss.pth')
                
            print(f'Epoch {epoch_i} | loss: {epoch_loss:.8f}')
            
            niter = epoch_i*len(self.train_loader)+batch_idx
            writer.add_scalar('Train/Loss', loss.data[0], niter)
            self.evaluate(epoch_i)
            
        # load and test
        self.net.load_state_dict(torch.load('models/MobileNet_CIFAR_test.pth'))
        print (f'Best Loss {best_loss} | Best')
        self.evaluate(epoch_i+1)
    
    def evaluate(self, epoch_i = None):
        total = 0
        correct = 0
        self.net.eval()
        self.best_acc = 0
        for batch_idx, (images, labels) in enumerate(self.test_loader):
            images = Variable(images).cuda()

            logits = self.net(images.cuda())
            
            _, predicted = torch.max(logits.data, 1)
            
            acc1, acc5 = accuracy(logits.cpu(), labels.cpu(), topk=(1, 5))
            if best_loss < float(acc1):
                self.best_acc = float(acc1)
                torch.save(self.net.state_dict(), 'models/MobileNet_CIFAR_best_acc.pth') 
            
            total += labels.size(0)
            correct += (predicted.cpu() == labels).sum()
        if epoch_i is not None:
            writer.add_scalar('Test/Acc@1', acc1, epoch_i)
            writer.add_scalar('Test/Acc@5', acc5, epoch_i)
        print(f'Top-1 Accuracy: {float(acc1)}')
        print(f'Top-5 Accuracy: {float(acc5)}')

In [ ]:
standard_solver = Solver(MobileNet_CIFAR, 'CIFAR10')

standard_solver.train()